<a href="https://colab.research.google.com/github/GavinButts/Fall-Data-Challenge-2022/blob/main/Code/Kevin/Ordinal%20-%20SEABSNT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib as plt
import numpy as np
import seaborn as sns
!pip install --upgrade --no-deps statsmodels
import scipy.stats as stats


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.9 MB 8.1 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.12.2
    Uninstalling statsmodels-0.12.2:
      Successfully uninstalled statsmodels-0.12.2


In [2]:
sheet_id = "11rno0SjsOyV1JlevH3q40UMm1s5e6m7g"
sheet_name = "curated_2019-required"

url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

df = pd.read_csv(url)


In [3]:
#create FOYESCNT variable that determines the amount of times each person said 'Yes' on the FO#### questions (ITEMS 72-74)
#Legend:  Number of 'Yes'

FOYESCNT = []      #variable that will be added to csv
yesCount = 0
for i in range(15500):
  if df['FOSTORY2X'][i] == 1:
    yesCount += 1
  if df['FOCRAFTS'][i] == 1:
    yesCount += 1
  if df['FOGAMES'][i] == 1:
    yesCount += 1
  if df['FOBUILDX'][i] == 1:
    yesCount += 1
  if df['FOSPORT'][i] == 1:
    yesCount += 1
  if df['FORESPON'][i] == 1:
    yesCount += 1
  if df['FOHISTX'][i] == 1:
    yesCount += 1
  if df['FOLIBRAYX'][i] == 1:
    yesCount += 1
  if df['FOBOOKSTX'][i] == 1:
    yesCount += 1
  if df['FODINNERX'][i] != 0:
    yesCount += 1
  FOYESCNT.append(yesCount)
  yesCount = 0

df["FOYESCNT"] = FOYESCNT
df.to_csv("sample.csv", index=False)  #adds variable to csv

#create FSYESCNT variable that determines the amount of times each person said 'Yes' on the FS#### questions (ITEMS 60)
#Legend:  Number of 'Yes'

FSYESCNT = []      #variable that will be added to csv
yesCount = 0
FSYESCNT_LMH = []

for i in range(15500):
  if df['FSSPORTX'][i] == 1:
    yesCount += 1
  if df['FSVOL'][i] == 1:
    yesCount += 1
  if df['FSMTNG'][i] == 1:
    yesCount += 1
  if df['FSPTMTNG'][i] == 1:
    yesCount += 1
  if df['FSATCNFN'][i] == 1:
    yesCount += 1
  if df['FSFUNDRS'][i] == 1:
    yesCount += 1
  if df['FSCOMMTE'][i] == 1:
    yesCount += 1
  if df['FSCOUNSLR'][i] == 1:
    yesCount += 1
  FSYESCNT.append(yesCount)
  if yesCount <= 3:
    FSYESCNT_LMH.append('1) LOW INVOLVMENT')
  elif yesCount > 3 and yesCount<=7:
    FSYESCNT_LMH.append('2) MEDIUM INVOLVEMENT')
  elif yesCount > 7:
    FSYESCNT_LMH.append('3) HIGH INVOLVEMENT')
  yesCount = 0

df["FSYESCNT"] = FSYESCNT
df["FSYESCNT_LMH"] = FSYESCNT_LMH

df.to_csv("sample.csv", index=False)  #adds variable to csv


In [4]:
# delete all rows for students who answered 5 or -1
'''
df_clean = []
for i in range(len(df)):
  if !(df.SEGRADES[i] == -1 or df.SEGRADES[i] == 5):
    df_clean.append(df[]) = df.drop(i)
'''


i = 0
L = len(df.SEGRADES)
while i < L:
  if ((df.SEGRADES[i] == -1) or (df.SEGRADES[i] == 5) or
      df.SEENJOY[i] == -1 or 
      df.FSNOTESX[i] == -1 or
      df.FSSPORTX[i] == -1 or
      df.FSPTMTNG[i] == -1 or
      df.FSCOUNSLR[i] == -1 or
      df.FSFUNDRS[i] == -1 or
      df.FSATCNFN[i] == -1 or
      df.SEABSNT[i] == -1):
    df = df.drop(i)
  i+=1


In [5]:
#Data Pre-Processing
from pandas.api.types import CategoricalDtype
cat_type = CategoricalDtype(categories=[1,2,3,4], ordered=True)
df["SEENJOY"] = df["SEENJOY"].astype(cat_type)
df["SEENJOY"].dtype

CategoricalDtype(categories=[1, 2, 3, 4], ordered=True)

In [6]:

from statsmodels.miscmodels.ordinal_model import OrderedModel
df

,BASMID,ALLGRADEX,EDCPUB,SCCHOICE,SPUBCHOIX,SCONSIDR,SCHLHRSWK,EINTNET,MOSTIMPT,INTNUM,...,HHPARN19_BRD,NUMSIBSX,PARGRADEX,RACEETH,INTACC,CENREG,ZIPLOCL,FOYESCNT,FSYESCNT,FSYESCNT_LMH
0,20191000012,12,1,2,3,2,4,4,-1,-1,...,1,1,3,4,1,4,11,6,3,1) LOW INVOLVMENT
2,20191000059,11,2,2,2,1,4,3,2,2,...,2,1,5,2,3,2,11,6,7,2) MEDIUM INVOLVEMENT
3,20191000070,6,1,1,1,1,4,4,-1,-1,...,2,1,2,3,1,2,31,10,8,3) HIGH INVOLVEMENT
4,20191000078,9,1,2,3,2,4,4,-1,-1,...,1,1,5,5,1,2,21,6,4,2) MEDIUM INVOLVEMENT
6,20191000149,8,1,1,1,2,4,4,-1,-1,...,1,2,3,3,1,2,41,5,2,1) LOW INVOLVMENT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15494,20191225467,12,1,1,1,2,4,4,-1,-1,...,2,0,3,1,1,3,42,3,4,2) MEDIUM INVOLVEMENT
15495,20191225472,7,1,2,2,2,4,4,-1,-1,...,1,2,4,4,1,3,23,5,4,2) MEDIUM INVOLVEMENT
15497,20191225477,12,1,1,3,2,4,4,-1,-1,...,1,2,2,3,1,1,13,7,5,2) MEDIUM INVOLVEMENT
15498,20191225479,13,1,2,2,1,4,4,-1,-1,...,2,0,4,1,3,4,11,8,7,2) MEDIUM INVOLVEMENT


In [20]:
#['HHPARN19X','FHPLACE','SEABSNT','FSYESCNT','SEGRADES','FOYESCNT','RACEETH','TTLHHINC']
x = ['ALLGRADEX','SCONSIDR','FCORDER','HDHEALTH','SEENJOY','SEGRADES','RACEETH','OWNRNTHB','SEFUTUREX','PARGRADEX',
     'FSPTMTNG',
     'FSNOTESX','FSCOUNSLR']
mod_prob = OrderedModel(df['SEABSNT'],
                        df[x],
                        distr='logit')
 
res_log = mod_prob.fit(method='bfgs')
res_log.summary()


Optimization terminated successfully.
         Current function value: 0.553497
         Iterations: 57
         Function evaluations: 59
         Gradient evaluations: 59


<class 'statsmodels.iolib.summary.Summary'>
"""
                             OrderedModel Results                             
==============================================================================
Dep. Variable:                SEABSNT   Log-Likelihood:                -7426.3
Model:                   OrderedModel   AIC:                         1.488e+04
Method:            Maximum Likelihood   BIC:                         1.500e+04
Date:                Tue, 01 Nov 2022                                         
Time:                        20:24:03                                         
No. Observations:               13417                                         
Df Residuals:                   13401                                         
Df Model:                          16                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ALLGRADEX      0.0375      0.007      5.199      0.000       0.023       0.052
SCONSIDR      -0.1122      0.049     -2.297      0.022      -0.208      -0.016
FCORDER        0.1094      0.031      3.560      0.000       0.049       0.170
HDHEALTH       0.4326      0.029     15.029      0.000       0.376       0.489
SEENJOY        0.3652      0.036     10.043      0.000       0.294       0.437
SEGRADES       0.2175      0.034      6.437      0.000       0.151       0.284
RACEETH       -0.0509      0.019     -2.640      0.008      -0.089      -0.013
OWNRNTHB       0.1962      0.048      4.049      0.000       0.101       0.291
SEFUTUREX     -0.0509      0.021     -2.388      0.017      -0.093      -0.009
PARGRADEX     -0.0953      0.023     -4.195      0.000      -0.140      -0.051
FSPTMTNG       0.1705      0.049      3.482      0.000       0.075       0.266
FSNOTESX      -0.2733      0.051     -5.309      0.000      -0.374      -0.172
FSCOUNSLR     -0.1916      0.050     -3.857      0.000      -0.289      -0.094
1/2            2.8269      0.245     11.537      0.000       2.347       3.307
2/3            0.5513      0.023     23.554      0.000       0.505       0.597
3/4            0.4244      0.051      8.291      0.000       0.324       0.525
==============================================================================
"""

In [21]:
predicted = res_log.model.predict(res_log.params, exog=df[x])
predicted


/usr/local/lib/python3.7/dist-packages/statsmodels/miscmodels/ordinal_model.py:419: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  xb = xb[:, None]


array([[0.88002666, 0.09649985, 0.01828842, 0.00518507],
       [0.61851032, 0.28340396, 0.07504814, 0.02303759],
       [0.91180835, 0.07142352, 0.01308392, 0.00368421],
       ...,
       [0.68914001, 0.23718416, 0.05672251, 0.01695332],
       [0.52093104, 0.33954155, 0.10556223, 0.03396518],
       [0.74936778, 0.19494416, 0.0430626 , 0.01262546]])

In [22]:
pred_choice = predicted.argmax(1)
print('Fraction of correct choice predictions')
print((np.asarray(df['SEENJOY'].values.codes) == pred_choice).mean())


Fraction of correct choice predictions
0.3536558097935455
